# Rolling window for arrays in NumPy

[Self-link](https://colab.research.google.com/drive/1WPwxovVxXi9YBrQVUavaSJMt7I0rwlNh)

**Disclaimer**: *author could be wrong!* If you see the error, please, write me at [foobar167@gmail.com](mailto:foobar167@gmail.com?subject=%20Rolling%20window%20error).

It is possible to implement a rolling window for NumPy arrays and images *without explicit cycles* in Python. As a result the speed of a such rolling window will be comparable to the speed of **C** programming language. Because NumPy library is implemented in the **C** programming language. It is in several thousand times faster than explicit Python `for` cycles.

  00. [Introduction](#introduction)
  00. [Rolling 1D window for ND array in Numpy](#1d)
  00. [Rolling 2D window for ND array in Numpy](#2d)
  00. [Rolling 3D window for ND array in Numpy](#3d)
  00. [Rolling MD window for ND array, where M ≤ N](#md)
  00. [Rolling MD window for ND array for any M and N](#md-extended)

## <a name="introduction">Introduction</a>
This article is an extension of [my answer](https://stackoverflow.com/a/46237736/7550928) on the StackOverflow. My first experiments with the rolling window are [here](https://github.com/foobar167/junkyard/blob/master/rolling_window.py) and [here](https://github.com/foobar167/junkyard/blob/master/rolling_window_advanced.py).

**Practical implementation** of the rolling 2D window for 2D array in NumPy is in the `roll` function of the [`logic_tools.py`](https://github.com/foobar167/junkyard/blob/master/manual_image_annotation1/polygon/logic_tools.py) file of the [Manual image annotation with polygons](https://github.com/foobar167/junkyard/tree/master/manual_image_annotation1) project.

Basics rolling window technique for 1D array is already explained [here](https://stackoverflow.com/a/7100681/7550928), [here](https://rigtorp.se/2011/01/01/rolling-statistics-numpy.html) and [here](https://stackoverflow.com/questions/6811183/rolling-window-for-1d-arrays-in-numpy).

To understand the topic, you must know what [strides](https://stackoverflow.com/a/53099870/7550928) are.

In [0]:
# Import necessary libraries
import numpy as np

## <a name="1d" />1. Rolling 1D window for ND array in Numpy

In [0]:
# Rolling 1D window for ND array
def roll(a,      # ND array
         b,      # rolling 1D window array
         dx=1):  # step size (horizontal)
    shape = a.shape[:-1] + (int((a.shape[-1] - b.shape[-1]) / dx) + 1,) + b.shape
    strides = a.strides[:-1] + (a.strides[-1] * dx,) + a.strides[-1:]
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

There are 2 major things for newly created array: **shape** and **strides**. Shape of the new array is created via shape of the input ND array and shape of the rolling 1D window. While strides are created only via strides of the input ND array (without rolling window).

Step size of the rolling window **dx** is equal to 1, 2, 3, etc. and always horizontal for 1D rolling window.

*Shape* consists from 3 terms:
  * `a.shape[:-1]` is the array shape remainder for ND arrays where `N > 1`. For `N == 1` array remainder will be equal to empty tuple `t == ()`, so it is not necessary for `N == 1`.
  * `(int((a.shape[-1] - b.shape[-1]) / dx) + 1,)` is the number of steps of the rolling 1D window over last dimension of the ND array.
  * `b.shape` is the shape of the rolling window.

*Strides* also consist from 3 terms:
  * `a.strides[:-1]` is the array strides remainder for ND array where `N > 1`. For `N == 1` array remainder will be equal to empty tuple `t == ()`, so it is not necessary for `N == 1`.
  * `(a.strides[-1] * dx,)` is the number of bytes between steps of the rolling window. For example, `int` array has 4 bytes stride between neighbour elements, so for step `dx == 2` it should be `4 * 2 = 8` bytes between steps of the rolling window.
  * `(a.strides[-1],)` is the stride between neighbour elements of the input ND array. For example, for `int` it should be equal 4 bytes or tuple `(4,)`.

  Function [`numpy.lib.stride_tricks.as_strided`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.lib.stride_tricks.as_strided.html) creates a view into the array with the given shape and strides.

In [34]:
# Tests
def show_results(a, b, dx=1):
    axis = a.ndim  # number of dimensions
    bool_array = np.all(roll(a, b, dx) == b, axis=axis)
    counts = np.count_nonzero(bool_array)
    coords = np.transpose(np.nonzero(bool_array)) * [1, dx]
    print("Found {counts} elements with coordinates:\n{coords}".format(
        counts=counts, coords=coords))

a = np.array([[ 0,  1,  2,  3,  4,  5],
              [ 7,  8,  7,  8, 10, 11],
              [13, 14, 13, 14,  7,  8],
              [19, 20, 19, 20, 13, 14],
              [24, 25, 26, 27, 19, 20]], dtype=np.int)

# Should find: 3 elements if dx == 1 or dx == 2
#              1 element  if dx == 3
#              2 elements if dx == 4
b1 = np.array([7, 8], dtype=np.int)
show_results(a, b1)
show_results(a, b1, 2)
show_results(a, b1, 3)
show_results(a, b1, 4)

# Should find: 1 element  if dx == 1
#              0 elements if dx == 2
b2 = np.array([8, 7], dtype=np.int)
print("----------")
show_results(a, b2)
show_results(a, b2, 2)

Found 3 elements with coordinates:
[[1 0]
 [1 2]
 [2 4]]
Found 3 elements with coordinates:
[[1 0]
 [1 2]
 [2 4]]
Found 1 elements with coordinates:
[[1 0]]
Found 2 elements with coordinates:
[[1 0]
 [2 4]]
----------
Found 1 elements with coordinates:
[[1 1]]
Found 0 elements with coordinates:
[]


## <a name="2d" />2. Rolling 2D window for ND array in Numpy
Example of practical implementation of the rolling 2D window for 2D array is to:
  * find smaller sub-image in the bigger image;
  * do a convolution operation in the artificial neural network;
  * apply filter of the artificial neural network or of the classical algorithm (Sobel, Gaussian or Blur filter) to the image.

In general it allows to make a *periodic operation* (comparison, convolution, subtraction, multiplication, some filter application, etc.) on a submatrix with some step size.

In [0]:
# Rolling 2D window for ND array
def roll(a,      # ND array
         b,      # rolling 2D window array
         dx=1,   # horizontal step, abscissa, number of columns
         dy=1):  # vertical step, ordinate, number of rows
    shape = a.shape[:-2] + \
            ((a.shape[-2] - b.shape[-2]) // dy + 1,) + \
            ((a.shape[-1] - b.shape[-1]) // dx + 1,) + \
            b.shape  # sausage-like shape with 2D cross-section
    strides = a.strides[:-2] + \
              (a.strides[-2] * dy,) + \
              (a.strides[-1] * dx,) + \
              a.strides[-2:]
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

There are also shape and strides like in [section one](#1d), but for rolling 2D window.

*Shape* consists from 4 terms: 3 terms like in [section one](#1d) and the 4th term is the number of vertical steps `(int((a.shape[0] - b.shape[0]) / dy) + 1,)` of the rolling 2D window.

We can replace:
```
    (int((a.shape[-1] - b.shape[-1]) / dx) + 1,)
```
with
```
    ((a.shape[-1] - b.shape[-1]) // dx + 1,)
```
because these two expressiona are equivalent.

*Strides* are also similar to [section one](#1d), but with additional stride `(a.strides[0] * dy,)` for vertical step of the rolling 2D window.

In [36]:
# Tests
def show_results(a, b, dx=1, dy=1):
    axis = a.ndim  # number of dimensions
    bool_array = np.all(
                   np.all(roll(a, b, dx, dy) == b, axis=axis),
                 axis=axis)  # np.all over 2 dimensions of the rolling 2D window
    counts = np.count_nonzero(bool_array)
    coords = np.transpose(np.nonzero(bool_array)) * [dy, dx]
    print("Found {counts} elements with coordinates:\n{coords}".format(
        counts=counts, coords=coords))

a = np.array([[ 0,  1,  2,  3,  4,  5],
              [ 7,  8,  7,  8, 10, 11],
              [13, 14, 13, 14,  7,  8],
              [19, 20, 19, 20, 13, 14],
              [24, 25, 26, 27, 19, 20]], dtype=np.int)

# Should find: 3 elements if dx == 1 or dx == 2
#              1 element  if dx == 1 and dy == 2
#              2 elements if dx == 4
b1 = np.array([[ 7,  8],
               [13, 14]], dtype=np.int)
show_results(a, b1)
show_results(a, b1, 2)
show_results(a, b1, 1, 2)
show_results(a, b1, 4)

# Should find: 1 element  if dx == 1
#              0 elements if dx == 2
b2 = np.array([[ 8,  7],
               [14, 13]], dtype=np.int)
print("----------")
show_results(a, b2)
show_results(a, b2, 2)

Found 3 elements with coordinates:
[[1 0]
 [1 2]
 [2 4]]
Found 3 elements with coordinates:
[[1 0]
 [1 2]
 [2 4]]
Found 1 elements with coordinates:
[[2 4]]
Found 2 elements with coordinates:
[[1 0]
 [2 4]]
----------
Found 1 elements with coordinates:
[[1 1]]
Found 0 elements with coordinates:
[]


## <a name="3d" />3. Rolling 3D window for ND array in Numpy
You can see the pattern for 1D and 2D rolling window. It is difficult to understand the algorithm in higher dimensions, however it is possible to implement rolling 3D window for ND array in NumPy according to the pattern and then test it.

With this practical implementation you can apply voxels over 3D image.

In [0]:
# Rolling 3D window for ND array
def roll(a,      # ND array
         b,      # rolling 2D window array
         dx=1,   # horizontal step, abscissa, number of columns
         dy=1,   # vertical step, ordinate, number of rows
         dz=1):  # transverse step, applicate, number of layers
    shape = a.shape[:-3] + \
            ((a.shape[-3] - b.shape[-3]) // dz + 1,) + \
            ((a.shape[-2] - b.shape[-2]) // dy + 1,) + \
            ((a.shape[-1] - b.shape[-1]) // dx + 1,) + \
            b.shape  # multidimensional "sausage" with 3D cross-section
    strides = a.strides[:-3] + \
              (a.strides[-3] * dz,) + \
              (a.strides[-2] * dy,) + \
              (a.strides[-1] * dx,) + \
              a.strides[-3:]
    #print('shape =', shape, " strides =", strides)  # for debugging
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [53]:
# Tests
def show_results(a, b, dx=1, dy=1, dz=1):
    axis = a.ndim  # number of dimensions == 3
    # np.all over 3 dimensions of the rolling 3D window
    bool_array = np.all(np.all(np.all(
            roll(a, b, dx, dy, dz) == b,
        axis=axis), axis=axis), axis=axis)
    counts = np.count_nonzero(bool_array)
    coords = np.transpose(np.nonzero(bool_array)) * [dz, dy, dx]
    print("Found {counts} elements with coordinates:\n{coords}".format(
        counts=counts, coords=coords))

a = np.array([[[ 0,  1,  2,  3,  4,  5],
               [ 7,  8,  7,  8, 10, 11],
               [13, 14, 13, 14,  7,  8],
               [19, 20, 19, 20, 13, 14],
               [24, 25, 26, 27, 19, 20]],

              [[ 1,  1,  1,  1,  1,  1],
               [ 2,  2,  2,  2,  2,  2],
               [ 3,  3,  3,  3,  2,  2],
               [ 4,  7,  8,  4,  3,  3],
               [ 5, 13, 14,  5,  5,  5]],

              [[ 1,  1,  1,  1,  1,  1],
               [ 2,  2,  2,  2,  2,  2],
               [ 3,  3,  3,  3,  3,  3],
               [ 4,  2,  2,  4,  4,  4],
               [ 5,  3,  3,  5,  5,  5]],], dtype=np.int)

# Should find: 4 elements if dx == 1, dy == 1, dz == 1 
#              3 elements if dx == 2, dy == 1, dz == 1
#              1 element  if dx == 1, dy == 3, dz == 1
b1 = np.array([[[ 7,  8],  [13, 14]],
               [[ 2,  2],  [ 3,  3]]], dtype=np.int)
show_results(a, b1)
show_results(a, b1, 2, 1, 1)
show_results(a, b1, 1, 3, 1)

# Should find: 1 element  if dx == 1
#              0 elements if dx == 2
b2 = np.array([[[ 8,  7],  [14, 13]],
               [[ 2,  2],  [ 3,  3]]], dtype=np.int)
print("----------")
show_results(a, b2)
show_results(a, b2, 2)

Found 4 elements with coordinates:
[[0 1 0]
 [0 1 2]
 [0 2 4]
 [1 3 1]]
Found 3 elements with coordinates:
[[0 1 0]
 [0 1 2]
 [0 2 4]]
Found 1 elements with coordinates:
[[1 3 1]]
----------
Found 1 elements with coordinates:
[[0 1 1]]
Found 0 elements with coordinates:
[]


## <a name="md" />4. Rolling MD window for ND array, where M ≤ N
Generalize the `roll` function for rolling MD window over ND array, where M ≤ N.

In [0]:
# Rolling MD window for ND array
def roll(a,        # ND array
         b,        # rolling MD window array
         d=None):  # steps array

    # Make several verifications
    m = b.ndim  # rolling window dimension
    if m > a.ndim:  # check if M ≤ N
        print("Error: rolling window dimensions is larger than the array dims")
        return None
    if d is None:  # steps are equal to 1 by default
        d = np.ones(m, dtype=np.uint32)
    elif d.ndim != 1 and d.size != m:
        print("Error: steps number should be equal to rolling window dimensions")
        return None
    elif not np.issubdtype(d.dtype, np.integer) or \
         not (d > 0).all():
        print("Error: steps must be integer and > 0")
        return None

    s = np.flip(d)  # flip the 1D array of step sizes
    sub = np.subtract(a.shape[-m:], b.shape[-m:])
    steps = tuple(np.divide(sub, s).astype(np.uint32) + 1)
    shape = a.shape[:-m] + steps + b.shape

    section = tuple(np.multiply(a.strides[-m:], s))
    strides = a.strides[:-m] + section + a.strides[-m:]

    #print('shape =', shape, " strides =", strides)  # for debugging
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [55]:
# Tests
def show_results(a, b, d=None):
    n = a.ndim  # array number of dimensions == N
    m = b.ndim  # rolling window dimensions == M
    if d is None:  # steps are equal to 1 by default
        d = np.ones(m, dtype=np.uint32)
    bool_array = roll(a, b, d) == b
    # np.all over N dimensions of the rolling MD window
    for i in range(n):
        bool_array = np.all(bool_array, axis=n)
    counts = np.count_nonzero(bool_array)
    s = np.flip(d)  # flip the 1D array of step sizes
    coords = np.transpose(np.nonzero(bool_array)) * s
    print("Found {counts} elements with coordinates:\n{coords}".format(
        counts=counts, coords=coords))

a = np.array([[[ 0,  1,  2,  3,  4,  5],
               [ 7,  8,  7,  8, 10, 11],
               [13, 14, 13, 14,  7,  8],
               [19, 20, 19, 20, 13, 14],
               [24, 25, 26, 27, 19, 20]],

              [[ 1,  1,  1,  1,  1,  1],
               [ 2,  2,  2,  2,  2,  2],
               [ 3,  3,  3,  3,  2,  2],
               [ 4,  7,  8,  4,  3,  3],
               [ 5, 13, 14,  5,  5,  5]],

              [[ 1,  1,  1,  1,  1,  1],
               [ 2,  2,  2,  2,  2,  2],
               [ 3,  3,  3,  3,  3,  3],
               [ 4,  2,  2,  4,  4,  4],
               [ 5,  3,  3,  5,  5,  5]],], dtype=np.int)

# Should find: 4 elements if dx == 1, dy == 1, dz == 1 
#              3 elements if dx == 2, dy == 1, dz == 1
#              1 element  if dx == 1, dy == 3, dz == 1
b1 = np.array([[[ 7,  8],  [13, 14]],
               [[ 2,  2],  [ 3,  3]]], dtype=np.int)
show_results(a, b1)
show_results(a, b1, np.array([2, 1, 1]))
show_results(a, b1, np.array([1, 3, 1]))

# Should find: 1 element  if dx == 1
#              0 elements if dx == 2
b2 = np.array([[[ 8,  7],  [14, 13]],
               [[ 2,  2],  [ 3,  3]]], dtype=np.int)
print("----------")
show_results(a, b2)
show_results(a, b2, np.array([2, 1, 1]))

Found 4 elements with coordinates:
[[0 1 0]
 [0 1 2]
 [0 2 4]
 [1 3 1]]
Found 3 elements with coordinates:
[[0 1 0]
 [0 1 2]
 [0 2 4]]
Found 1 elements with coordinates:
[[1 3 1]]
----------
Found 1 elements with coordinates:
[[0 1 1]]
Found 0 elements with coordinates:
[]


## <a name="md-extended" />5. Rolling MD window for ND array for any M and N
Can we roll MD window for ND arrays, where M > N? Actually, we can, but only part of the rolling window which **intersects** with ND array.

In [0]:
# Rolling MD window for ND array


In [0]:
# Tests
